In [73]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft,dct
import librosa
from scipy import signal
import soundfile as sf
from librosa.display import specshow
import glob

In [ ]:
carTrain = glob.glob("cars/train/*.wav")
carTest = glob.glob("cars/test/*.wav")

tramTrain = glob.glob("trams/train/*.wav")
tramTest = glob.glob("trams/test/*.wav")

dataset=[]
labels = []
